In [1]:
import os

import tensorflow as tf

/home/sinho/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv('data/fashion_train.csv')

In [6]:
df_train_X = df_train.iloc[:, 1:]
df_train_y = df_train.iloc[:,[0]]

In [7]:
df_train_X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,3,...,23,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,141,73,0,0,107,63,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,35,7,14,0,0,0


In [8]:
train_X = df_train_X

In [9]:
train_y = pd.get_dummies(df_train_y,columns=['label'],sparse=True)

In [10]:
df_test = pd.read_csv('data/fashion_test.csv')

In [11]:
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,1,0,0,0,0,0,0,0,0,0,...,208,174,128,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,1,0,1,...,49,40,3,0,40,66,11,0,0,0
2,8,0,0,0,0,0,0,0,0,0,...,128,126,126,120,129,75,0,0,0,0
3,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,123,145,152,206,58,0,0,0,0,0


In [12]:
df_test_X = df_test.iloc[:,1:]
df_test_y = df_test.iloc[:,[0]]

In [13]:
test_X = df_test_X
test_y = pd.get_dummies(df_test_y, columns=['label'], sparse = True)

ref)https://www.quora.com/What-is-the-best-way-to-read-data-into-Tensorflow

In [14]:
import os

fashion_train = pd.read_csv("./data/fashion_train.csv").values
fashion_test = pd.read_csv("./data/fashion_test.csv").values

In [15]:
def make_TFRecordWriter(csv, name):
    with tf.python_io.TFRecordWriter("{}.tfrecord".format(name)) as tf_writer:
        for row in csv:
            #file formate : feature 1…..feature n, label
            features, label = row[-1:], row[0]
            example = tf.train.Example()
            example.features.feature["features"].float_list.value.extend(features)
            example.features.feature["label"].int64_list.value.append(label)
            tf_writer.write(example.SerializeToString())

In [16]:
make_TFRecordWriter(fashion_train, 'fashion_train')
make_TFRecordWriter(fashion_test, 'fashion_test')

In [3]:
tf.reset_default_graph()

def parser(serialized_example):
    features = {
        'label' : tf.FixedLenFeature([1], tf.int64),
        'img' : tf.FixedLenFeature([28*28], tf.int64)
    }
    parsed_feature = tf.parse_single_example(serialized_example, features)
    label = parsed_feature['label']
    img = parsed_feature['img']
    return label, img

train_name = './data/fashion_train.tfrecord'
train_dataset = tf.data.TFRecordDataset(train_name).map(parser).batch(1).shuffle(9999999999999)

test_name = './data/fashion_test.tfrecord'
test_dataset = tf.data.TFRecordDataset(test_name).map(parser).batch(1).shuffle(9999999999999)

train_itr = tf.contrib.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
train_label, train_img = train_itr.get_next()

train_img = tf.reshape(train_img, [-1, 28,28 ,1])
train_img = tf.cast(train_img, tf.float32)

train_label = tf.reshape(train_label, [-1])
train_label = tf.one_hot(train_label, depth=10, axis=-1 , dtype =tf.float32)

test_itr = tf.contrib.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)
test_label, test_img = test_itr.get_next()

test_img = tf.reshape(test_img, [-1, 28,28 ,1])
test_img = tf.cast(test_img, tf.float32)

test_label = tf.reshape(test_label, [-1])
test_label = tf.one_hot(test_label, depth=10, axis=-1 , dtype =tf.float32)

training_init_op = train_itr.make_initializer(train_dataset)
testing_init_op = test_itr.make_initializer(test_dataset)

def cnn_model(x, activation, drop_prob, training , reuse):

    conv1 = tf.layers.conv2d(x, 
                                         filters = 32,
                                         kernel_size = 3, 
                                         activation = activation,
                                         padding = 'same', 
                                         reuse = reuse,
                                         name = 'conv1')

    print('conv1 shape: ',conv1.shape)

    conv2 = tf.layers.conv2d(conv1, 
                                         filters = 64, 
                                         kernel_size = 3, 
                                         activation = activation,
                                         padding = 'same', 
                                         reuse = reuse,
                                         name = 'conv2')

    print('conv2 shape: ',conv2.shape)

    pool2 = tf.layers.max_pooling2d(conv2, pool_size = 2, strides= 2)

    print('pool2 shape: ',pool2.shape)


    flat = tf.layers.flatten(pool2)

    fc1 = tf.layers.dense(flat, 300, reuse = reuse , name= 'fc1')
    
    dropout1 =tf.layers.dropout(fc1, drop_prob , training = training )
    
    fc2 = tf.layers.dense(dropout1, 100,  reuse = reuse , name= 'fc2')
    
    dropout2 =tf.layers.dropout(fc2, drop_prob ,  training = training )
    
    logits = tf.layers.dense(dropout2 , 10 , reuse = reuse , name = 'out')
    
    return logits

train_out = cnn_model(train_img, activation =tf.nn.relu , drop_prob = 0.5 , training = True , reuse = False )
test_out = cnn_model(test_img, activation =tf.nn.relu , drop_prob = 0.5 , training = True , reuse = True )

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = train_label, logits = train_out))
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

pred = tf.nn.softmax(test_out)

accuracy = tf.metrics.accuracy(tf.argmax(test_label, axis=1), tf.argmax(pred, axis=1))

tf.summary.scalar('loss', loss)
for v in tf.trainable_variables():
    tf.summary.histogram('filter {}'.format(v.name), v)
    
merged = tf.summary.merge_all()


config = tf.ConfigProto()
config.log_device_placement = True

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for i in range(100):
        _, _loss, _acc, _summ = sess.run([optimizer, loss, accuracy, merged])
        
        print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))

conv1 shape:  (?, 28, 28, 32)
conv2 shape:  (?, 28, 28, 64)
pool2 shape:  (?, 14, 14, 64)
conv1 shape:  (?, 28, 28, 32)
conv2 shape:  (?, 28, 28, 64)
pool2 shape:  (?, 14, 14, 64)
INFO:tensorflow:Summary name filter conv1/kernel:0 is illegal; using filter_conv1/kernel_0 instead.
INFO:tensorflow:Summary name filter conv1/bias:0 is illegal; using filter_conv1/bias_0 instead.
INFO:tensorflow:Summary name filter conv2/kernel:0 is illegal; using filter_conv2/kernel_0 instead.
INFO:tensorflow:Summary name filter conv2/bias:0 is illegal; using filter_conv2/bias_0 instead.
INFO:tensorflow:Summary name filter fc1/kernel:0 is illegal; using filter_fc1/kernel_0 instead.
INFO:tensorflow:Summary name filter fc1/bias:0 is illegal; using filter_fc1/bias_0 instead.
INFO:tensorflow:Summary name filter fc2/kernel:0 is illegal; using filter_fc2/kernel_0 instead.
INFO:tensorflow:Summary name filter fc2/bias:0 is illegal; using filter_fc2/bias_0 instead.
INFO:tensorflow:Summary name filter out/kernel:0 is 

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[Node: IteratorGetNext_1 = IteratorGetNext[output_shapes=[[?,1], [?,784]], output_types=[DT_INT64, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator_1)]]

Caused by op 'IteratorGetNext_1', defined at:
  File "/home/sinho/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sinho/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sinho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sinho/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sinho/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 47

In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

fashion_mnist = input_data.read_data_sets('input/data', one_hot=True)

Extracting input/data/train-images-idx3-ubyte.gz
Extracting input/data/train-labels-idx1-ubyte.gz
Extracting input/data/t10k-images-idx3-ubyte.gz
Extracting input/data/t10k-labels-idx1-ubyte.gz


In [5]:

# Shapes of training set
print("Training set (images) shape: {shape}".format(shape=fashion_mnist.train.images.shape))
print("Training set (labels) shape: {shape}".format(shape=fashion_mnist.train.labels.shape))

# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=fashion_mnist.test.images.shape))
print("Test set (labels) shape: {shape}".format(shape=fashion_mnist.test.labels.shape))

Training set (images) shape: (55000, 784)
Training set (labels) shape: (55000, 10)
Test set (images) shape: (10000, 784)
Test set (labels) shape: (10000, 10)


In [6]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation= tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10 , activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_size = 100
    total_batch = int(fashion_mnist.train.num_examples / batch_size)
    
    for epoch in range(30):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            _,cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, 
                                                                                        Y: batch_ys,
                                                                                        is_training: True})
            total_cost += cost_val
        print('Epoch: ', '%04d' % (epoch +1),
                 ' Avg. cost = ', '{:.4f}'.format(total_cost / total_batch))
    print('-'*30)
    print('최적화 완료!')
    print('-'*30)
    print('Test')
    is_corrent = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_corrent, tf.float32))
    print('정확도 : ', sess.run(accuracy, feed_dict= {X: fashion_mnist.test.images.reshape(
                                                                            -1, 28, 28, 1),
                                                                          Y: fashion_mnist.test.labels,
                                                                          is_training: False}))

Epoch:  0001  Avg. cost =  0.1786
Epoch:  0002  Avg. cost =  0.0488
Epoch:  0003  Avg. cost =  0.0326
Epoch:  0004  Avg. cost =  0.0218
Epoch:  0005  Avg. cost =  0.0159
Epoch:  0006  Avg. cost =  0.0140
Epoch:  0007  Avg. cost =  0.0122
Epoch:  0008  Avg. cost =  0.0107
Epoch:  0009  Avg. cost =  0.0073
Epoch:  0010  Avg. cost =  0.0079
Epoch:  0011  Avg. cost =  0.0070
Epoch:  0012  Avg. cost =  0.0075
Epoch:  0013  Avg. cost =  0.0062
Epoch:  0014  Avg. cost =  0.0064
Epoch:  0015  Avg. cost =  0.0046
Epoch:  0016  Avg. cost =  0.0025
Epoch:  0017  Avg. cost =  0.0046
Epoch:  0018  Avg. cost =  0.0071
Epoch:  0019  Avg. cost =  0.0064
Epoch:  0020  Avg. cost =  0.0047
Epoch:  0021  Avg. cost =  0.0037
Epoch:  0022  Avg. cost =  0.0032
Epoch:  0023  Avg. cost =  0.0034
Epoch:  0024  Avg. cost =  0.0061
Epoch:  0025  Avg. cost =  0.0049
Epoch:  0026  Avg. cost =  0.0020
Epoch:  0027  Avg. cost =  0.0023
Epoch:  0028  Avg. cost =  0.0071
Epoch:  0029  Avg. cost =  0.0033
Epoch:  0030  